In [0]:
import sys
sys.path.append(f'/Workspace/Users/{dbutils.widgets.get("account")}/music_data_lake/src/lib')

import utils
import ingestors

In [0]:
config = {
    "catalog": dbutils.widgets.get("catalog"),
    "schema": dbutils.widgets.get("schema"),
    "tablename": dbutils.widgets.get("tablename"),
    "primary_key": dbutils.widgets.get("primary_key"),
    "timestamp_field": dbutils.widgets.get("timestamp_field"),
    "id_field_old": dbutils.widgets.get("id_field_old")
}

In [0]:
remove_checkpoint = False

if not utils.table_exists(spark, config["catalog"], config["schema"], config["tablename"]):

    print("Criando a tabela", config["tablename"])
    query = utils.import_query(f"{config['tablename']}.sql")
    (spark.sql(query)
          .write
          .format("delta")
          .mode("overwrite")
          .option("overwriteSchema", "true")
          .saveAsTable(f"{config['catalog']}.{config['schema']}.{config['tablename']}"))
    
    remove_checkpoint = True

In [0]:
print("Iniciando CDF...")

ingest = ingestors.SilverIngestor(spark=spark,
                               config=config)

if remove_checkpoint:
    dbutils.fs.rm(ingest.checkpoint_location, True)

stream = ingest.run()
print("Ok.")